In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
finance_news = pd.read_excel('finance_data/SmoothNLP专栏资讯数据集样本10k.xlsx')
finance_news = finance_news.drop(columns = ['Unnamed: 0'])

In [4]:
finance_news.columns

Index(['title', 'content', 'pub_ts'], dtype='object')

In [65]:
finance_news['title'].iloc[0]

'引入前美团地推大将，每日优鲜便利购如何上演“攻守道”?'

In [62]:
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize

In [64]:
summarize(finance_news['content'].iloc[0])

'在新零售业态当中，无人货架启动和运营成本貌似最低，主要面向2亿白领人群的上班时间，是新的流量价值洼地。因此无人货架成为新零售大潮中最先火起来的业态，半年多时间已有50多玩家入局：一类是创业玩家，以小e微店、猩便利、果小美、哈米科技为代表；一类是原有业务延展的创业玩家，以每日优鲜便利购、饿了么NOW、便利蜂为代表，多数在17年6月到9月入局；一类是巨头玩家，17年11月到12月入局，如有京东到家智能柜、顺丰丰e足食、阿里美的小卖柜。\n先做规模再谈盈亏是互联网的典型路径。点位之争，尤其是优质点位之争是无人货架规模化发展的第一步，也决定着终局时的位置。\n刘澍的加盟，将帮助每日优鲜便利购迅速打造出一支具备极强执行力的直销铁军，同时带来发展战略和执行策略上的经验补充，助力便利购快速占据优势局面。\n在这个基础上，做好基于用户需求的产品品类更新、“千架千面”同样重要，考验对消费者理解、供应链支撑及产品选择的能力，每日优鲜便利购拥有业务基础和大数据能力，更具备优势。未来和腾讯数据、平台打通后，可带来大数据分析+人工智能技术业落地，为用户精准的提供高频消费产品。'

In [28]:
import jieba
jieba.enable_paddle()

Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......


In [29]:
tokens = jieba.cut("所以，整个过程有点像勘探。首先是有人发起，然后有千百人买了镐和铁锹跟进。有人会满载而归，但看走眼的人也会空手回来。"
                   ,cut_all=False)

In [69]:
from zhon.hanzi import punctuation as punctuation_zh
from string import punctuation
from words_utils import tokenization as finance_token
import re

In [83]:
def clean_text(content):
    content = re.sub('\xa0 ?|\u3000+', '', content)
    content = re.sub(' ?\n+', '\n', content)
    content = content.strip('\n')
    return content

In [31]:
stop_words = finance_token.read_stop_words()
tokenizer = finance_token.JiebaTokenizer(stop_words, [punctuation, punctuation_zh], vocab = 'finance_data/vocab')

In [57]:
finance_news['title'].iloc[3]

'王石遭“疯抢”！为什么那么多公司想聘请他当董事长？'

In [58]:
finance_news['content'].iloc[3]

'\n\xa0 \xa0 \xa0 67岁的王石，终究还是回到了他熟悉的商业领域这一战场。在此前一段时间，王石似乎是习惯性地过起了“佛系”生活。在之前的一年时间中，王石身上的标签是“深潜”、“划艇”、“公益”、“艺术”、“生态”。王石做网红大叔好像“上瘾”了，褪去那种在商界叱咤风云的霸气。\n\xa0 \xa0 \xa0 但王石终究不是一个会耐得住寂寞的人，今年成为了两家公司的联席董事长。值得一提的是，华大集团、远大集团的董事长都是王石的好朋友，因此王石才答应成为联席董事长。至于其他邀请自己的公司，王石通通都拒绝了。不过面对这样的“疯抢”，事实上也证明着王石有着自身独特的魅力。\n\xa0 \xa0 \xa0 有句老话说得好，“人的名，树的影”。执掌万科多年的王石，早就成为商界的标志性人物。再说了，王石还是8848手机的代言人！唔，也算是手机圈的“另类网红”了。而且能够在万科这样一个巨型平台上游刃有余地奋斗那么多年，自然能够反映出王石有着足够强的能力。\n\n\xa0 \xa0 \xa0 当然，更被其他公司注重的是王石的人脉和资源等。在商界有着极大影响力的王石，哪怕不负责公司的任何具体事务，只要挂个名就能让这家公司瞬间成为关注的焦点。\n\xa0 \xa0 \xa0 比如就在王石宣布成为华大集团联席董事长后，华大集团的股价当天就一路飙涨，甚至实现连续数天的狂涨势头。如果王石动用自己的人脉和资源，更有可能为公司插上腾飞的翅膀。看来，王石本身的影响力和“朋友圈”是他受到“疯抢”的根本原因。\n\n\xa0 \xa0 \xa0 但必须要指出的是，王石最熟悉的还是地产圈。而华大集团和远大集团的主要业务是有关生物科技、环保方面，王石究竟能够发挥出多少能力还有待商榷。比如华大集团表示王石将发挥其在经营管理、制度建设、商业运作等方面的丰富经验和卓越能力，协助董事长管理集团事宜。不过王石真能像管理地产集团那样，去管理生物科技公司吗？我们还不得而知。\n\xa0 \xa0 \xa0 而就目前来看，王石在这些公司似乎还偏向于“玄学”意味的管理模式。就像王石提到的那样，“无论是代表传统的还是代表未来的企业，还有我的位置，很多基本的逻辑和道理是一样的”。这是不是意味着王石并没有真正的全面突破，还是会延续既有的管理经验呢？结果到底如何，一切都交给时间去验证吧！（科技新发现 康斯坦丁/文）\n

In [5]:
all_token_length = []
with open('finance_data/data.txt','r', encoding='UTF-8') as file:
    for line in file:
        line = line.strip()
        if len(line) == 0:
            continue
        tokens = tokenizer.tokenize(line)
        all_token_length.append(len(tokens))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.584 seconds.
Prefix dict has been built successfully.


In [11]:
all_token_length = pd.DataFrame(all_token_length)

In [13]:
all_token_length.describe()

,0
count,296933.000000
mean,25.683282
std,22.148801
min,0.000000
25%,10.000000
50%,23.000000
75%,37.000000
max,1902.000000


In [38]:
all_token_length.quantile(q = 0.80)

0    40.0
Name: 0.8, dtype: float64